In [1]:
 ## Use this for Azure AD authentication
 from msrestazure.azure_active_directory import AADTokenCredentials

## Required for Data Lake Storage Gen1 account management
 from azure.mgmt.datalake.store import DataLakeStoreAccountManagementClient
 from azure.mgmt.datalake.store.models import DataLakeStoreAccount

 ## Required for Data Lake Storage Gen1 filesystem management
 from azure.datalake.store import core, lib, multithread

 # Common Azure imports
import adal
 from azure.mgmt.resource.resources import ResourceManagementClient
 from azure.mgmt.resource.resources.models import ResourceGroup

 ## Use these as needed for your application
 import logging, getpass, pprint, uuid, time

# authority_host_uri = 'https://login.microsoftonline.com'
# tenant = '2dff09ac-2b3b-4182-9953-2b548e0d0b39'
# authority_uri = authority_host_uri + '/' + tenant
# RESOURCE = 'https://management.core.windows.net/'
# client_id = '909c65ae-18db-4bb9-bbea-9a3c12eef354'
# client_secret = '-47-m~UGFWxa8c_-AUe31SNuW3vFs8oFFS'

# context = adal.AuthenticationContext(authority_uri, api_version=None)
# mgmt_token = context.acquire_token_with_client_credentials(RESOURCE, client_id, client_secret)
# armCreds = AADTokenCredentials(mgmt_token, client_id, resource=RESOURCE)

tenant = '2dff09ac-2b3b-4182-9953-2b548e0d0b39'
RESOURCE = 'https://datalake.azure.net/'
client_id = '909c65ae-18db-4bb9-bbea-9a3c12eef354'
client_secret = '-47-m~UGFWxa8c_-AUe31SNuW3vFs8oFFS'

adlCreds = lib.auth(tenant_id = tenant,
                client_secret = client_secret,
                client_id = client_id,
                resource = RESOURCE)

## Declare variables
subscriptionId = 'd2e64416-5057-45ac-a5bc-023f5107baf8'
adlsAccountName = 'uitstorage'

## Create a filesystem client object
adlsFileSystemClient = core.AzureDLFileSystem(adlCreds, store_name=adlsAccountName)


In [2]:
import numpy as np
import math
import csv
import glob
import imageio
import numpy as np
from mlxtend.image import extract_face_landmarks

In [3]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
  A = distance.euclidean(mouth[15], mouth[21])
  C = distance.euclidean(mouth[12], mouth[18])
  mar = (A ) / (C)
  return mar

def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

def getFrame(sec):
    start = 180000
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [4]:
from scipy.spatial import distance
import cv2
import os
dataSet = {"Fold4_part2":[48],"Fold5_part1":[51,54],"Fold5_part2":[55,60]}
labelList = [0,5,10]
azurePath = "dataset/"
clientPath = "./data/"
for i in dataSet:
    for j in dataSet[i]:
        for label in labelList:
            data = []
            labels = []
            rPath = azurePath + i + '/' + str(j) + '/' + str(label) + ".mp4"
            lPath = clientPath + str(i) + '/' + str(label) + ".mp4"
            ## Download a file
            multithread.ADLDownloader(adlsFileSystemClient, lpath=lPath, rpath=rPath, nthreads=64, overwrite=True, buffersize=4194304, blocksize=4194304)
            print(rPath)
            vidcap = cv2.VideoCapture(lPath)
            sec = 0
            frameRate = 1
            success, image  = getFrame(sec)
            count = 0
            while success and count < 240: 
              landmarks = extract_face_landmarks(image)
              if landmarks is not None and sum(sum(landmarks)) != 0:
                  count += 1
                  data.append(landmarks)
                  labels.append([label])
                  sec = sec + frameRate
                  sec = round(sec, 2)
                  success, image = getFrame(sec)
                  print(count)
              else:  
                  sec = sec + frameRate
                  sec = round(sec, 2)
                  success, image = getFrame(sec)
                  print("not detected")
            data = np.array(data)
            labels = np.array(labels)
            features = []
            for d in data:
              eye = d[36:68]
              ear = eye_aspect_ratio(eye)
              mar = mouth_aspect_ratio(eye)
              cir = circularity(eye)
              mouth_eye = mouth_over_eye(eye)
              features.append([ear, mar, cir, mouth_eye])
            features = np.array(features)
            np.save(open(clientPath + str(j) + '/' + str(label) + '_Data.npy', 'wb'),data)
            np.save(open(clientPath + str(j) + '/' + str(label) + '_Feature.npy', 'wb'),features)
            np.save(open(clientPath + str(j) + '/' + str(label) + '_Label.npy', 'wb'),labels)
            np.savetxt(clientPath + str(j) + '/' + str(label) + "_Feature.csv", features, delimiter = ",")
            np.savetxt(clientPath + str(j) + '/' + str(label) + "_Label.csv", labels, delimiter = ",")
            vidcap.release()
            cv2.destroyAllWindows()
            if os.path.exists(lPath):
              os.remove(lPath)

dataset/Fold4_part2/48/0.mp4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
dataset/Fold4_part2/48/5.mp4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


/usr/local/lib/python3.6/dist-packages/mlxtend/image/extract_face_landmarks.py:66: UserWarning: No face detected.
  warnings.warn('No face detected.')


not detected
not detected
55
not detected
56
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
57
58
not detected
59
not detected
not detected
not detected
not detected
60
61
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
62
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
n

166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
dataset/Fold5_part2/55/5.mp4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
not detected
not detected
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
1